<a href="https://colab.research.google.com/github/Mu-niu13/Emotion-Analysis/blob/main/lib/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up Connection to Github

In [1]:
## install git and configure(so we know who commit the code)
# !apt-get install git
# !git config --global user.name "USER_NAME"
# !git config --global user.email "EMAIL"

# access google drive
# from google.colab import drive
# drive.mount('/content/drive')

# clone repo
!git clone https://github.com/Mu-niu13/Emotion-Analysis.git
%cd Emotion-Analysis

# set up personal access for push/pull
from getpass import getpass
token = getpass('Enter your GitHub PAT:')
!git remote set-url origin https://{token}@github.com/Mu-niu13/Emotion-Analysis.git

Cloning into 'Emotion-Analysis'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 110 (delta 67), reused 59 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 11.94 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/Emotion-Analysis
Enter your GitHub PAT:··········


In [ ]:
!make install

Installing dependencies...
# Install python3-venv if not present (specific to Debian/Ubuntu systems like Colab)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelea

## Model Training

#### 1. Set Up

In [41]:
# import libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

# import custom modules
from lib.data_preprocessing import load_data, get_label_columns, create_data_loader
from lib.model import EmotionClassifier
from lib.train import train_epoch
from lib.evaluate import eval_model

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


#### 2. Training Loop

In [3]:
# load data
df = load_data('/content/Emotion-Analysis/data/merged_filtered_data.csv')

# get label columns
label_columns = get_label_columns(df)
n_classes = len(label_columns)

# train val split
df_train, df_val = train_test_split(df, test_size=0.1, random_state=42)

# init tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# create data loaders
MAX_LEN = 150
BATCH_SIZE = 64

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, label_columns)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, label_columns)

# init model
model = EmotionClassifier(n_classes=n_classes, model_name='bert-base-uncased')
model = model.to(device)

# define optimizer with parameter groups
optimizer = AdamW([
    {'params': model.model.parameters(), 'lr': 2e-5},
    {'params': model.layer_norm.parameters(), 'lr': 1e-4},
    {'params': model.fc1.parameters(), 'lr': 1e-4},
    {'params': model.fc2.parameters(), 'lr': 1e-4},
    {'params': model.fc3.parameters(), 'lr': 1e-4},
], weight_decay=1e-2, correct_bias=False)

# define total steps and warmup steps
EPOCHS = 10
total_steps = len(train_data_loader) * EPOCHS
warmup_steps = int(0.1 * total_steps)

# set scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# define loss
loss_fn = nn.CrossEntropyLoss().to(device)

# enable Automatic Mixed Precision
scaler = torch.cuda.amp.GradScaler()

# set up early stopping
best_val_loss = float('inf')
epochs_no_improve = 0
n_epochs_stop = 3
best_model_state = None

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')

    # training
    train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        scaler
    )
    print(f'Train loss {train_loss}')

    # validation
    val_loss, outputs, targets = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device
    )
    print(f'Validation loss {val_loss}')

    # compute metrics
    outputs = torch.softmax(outputs, dim=1)
    outputs_np = outputs.cpu().numpy()
    targets_np = targets.cpu().numpy()

    max_prob_classes = outputs_np.argmax(axis=1)
    true_classes = targets_np

    overall_accuracy = (max_prob_classes == true_classes).mean()
    print(f'Overall Accuracy: {overall_accuracy:.4f}')

    report = classification_report(
        true_classes,
        max_prob_classes,
        target_names=label_columns,
        zero_division=0
    )
    print(report)

    # early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        best_model_state = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= n_epochs_stop:
            print('Early stopping!')
            model.load_state_dict(best_model_state)
            torch.save(model.state_dict(), 'best_model_state.bin')
            break

# save best model after training
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    torch.save(model.state_dict(), 'best_model_state.bin')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Validation loss 1.5487562502347505
Overall Accuracy: 0.4651
                      precision    recall  f1-score   support

             neutral       0.55      0.43      0.49      5478
positive recognition       0.48      0.54      0.51      3399
               anger       0.39      0.51      0.44      2018
             sadness       0.40      0.40      0.40       690
             anxiety       0.38      0.47      0.42       361
              regret       0.33      0.26      0.29       773
           happiness       0.57      0.59      0.58      3423
          discomfort       0.35      0.36      0.36      1969
           affection       0.46      0.46      0.46       950
           curiosity       0.35      0.45      0.39       890
            surprise       0.34      0.24      0.28       831

            accuracy                           0.47     20782
           macro avg       0.42      0.43      0.42     20782
        weighted avg       0.47      0.47      0.46     20782

Epoch 7

  0%|          | 0/2923 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/content/Emotion-Analysis/lib/train.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2923/2923 [19:31<00:00,  2.49it/s]


Train loss 1.21364694228125


100%|██████████| 325/325 [02:52<00:00,  1.89it/s]


Validation loss 1.5835807433495155
Overall Accuracy: 0.4606
                      precision    recall  f1-score   support

             neutral       0.54      0.45      0.49      5478
positive recognition       0.48      0.53      0.51      3399
               anger       0.40      0.46      0.43      2018
             sadness       0.38      0.42      0.40       690
             anxiety       0.40      0.45      0.42       361
              regret       0.28      0.28      0.28       773
           happiness       0.58      0.59      0.58      3423
          discomfort       0.35      0.35      0.35      1969
           affection       0.41      0.50      0.45       950
           curiosity       0.34      0.39      0.36       890
            surprise       0.33      0.23      0.27       831

            accuracy                           0.46     20782
           macro avg       0.41      0.42      0.41     20782
        weighted avg       0.46      0.46      0.46     20782

Early s

#### 3. Performance on "Read" Data

In [46]:
def evaluate_on_gpt_data(model, data_loader, device):
    model.eval()
    all_outputs = []
    all_labels = []
    texts = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            all_outputs.append(outputs)
            all_labels.append(labels)

    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)

    return all_outputs, all_labels, texts

In [1]:

n_classes = 11  # Update with the actual number of classes
model = EmotionClassifier(n_classes=n_classes, model_name='bert-base-uncased')
model = model.to(device)
model.load_state_dict(torch.load('best_model_state.bin'))
model.eval()

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load GPT data
gpt_df = load_data('/content/Emotion-Analysis/data/GPT_generated_emotion_data.csv')

# Get label columns in the correct order
label_columns = [col for col in gpt_df.columns if col not in ['id', 'text', 'example_very_unclear']]

# Create a label mapping dictionary
label_mapping = {index: label for index, label in enumerate(label_columns)}

# DataLoader for GPT data
MAX_LEN = 150
BATCH_SIZE = 32


def create_data_loader(df, tokenizer, max_len, batch_size, label_columns):
    ds = EmotionDataset(
        dataframe=df,
        tokenizer=tokenizer,
        max_len=max_len,
        label_columns=label_columns
    )

    return torch.utils.data.DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,    # Ensure data order is maintained
        num_workers=0
    )

gpt_data_loader = create_data_loader(gpt_df, tokenizer, MAX_LEN, BATCH_SIZE, label_columns)

# Corrected evaluate_on_gpt_data function
def evaluate_on_gpt_data(model, data_loader, device):
    model.eval()
    all_outputs = []
    all_labels = []
    texts = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            # Verify that 'text' is included in the batch
            print(batch.keys())  # Should output: dict_keys(['input_ids', 'attention_mask', 'labels', 'text'])
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            all_outputs.append(outputs)
            all_labels.append(labels)

    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)

    return all_outputs, all_labels

# Evaluate model on GPT data
outputs, labels, texts = evaluate_on_gpt_data(model, gpt_data_loader, device)

# Get predictions
outputs = torch.softmax(outputs, dim=1)
outputs_np = outputs.cpu().numpy()
labels_np = labels.cpu().numpy()

# Predicted classes
predicted_classes = outputs_np.argmax(axis=1)
true_classes = labels_np

# Overall accuracy
accuracy = (predicted_classes == true_classes).mean()
print(f'Accuracy on GPT-generated data: {accuracy:.4f}')

# Misclassified examples
misclassified_indices = np.where(predicted_classes != true_classes)[0]
misclassified_texts = [gpt_df['text'].to_list()[i] for i in misclassified_indices]
misclassified_true_labels = [label_mapping[int(true_classes[i])] for i in misclassified_indices]
misclassified_predicted_labels = [label_mapping[int(predicted_classes[i])] for i in misclassified_indices]

misclassified_df = pd.DataFrame({
    'text': misclassified_texts,
    'true_label': misclassified_true_labels,
    'predicted_label': misclassified_predicted_labels
})

print(f'Number of misclassified examples: {len(misclassified_df)}')
print("Misclassified examples:")
misclassified_df

NameError: name 'EmotionClassifier' is not defined